In [1]:
import pandas as pd
import sqlite3
from datetime import datetime as dt
import os

In [2]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Exception as e:
        print(e)
 
    return conn

In [2]:
path = '/home/aleksandr/'

In [4]:
con = create_connection(path + 'ertelecom_base.db')

In [5]:
cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('cat',), ('result',)]


Extract first level domains

In [6]:
import tldextract as tld
def get_top_level_domain(domain):
    domain = tld.extract(domain)
    domain = '.'.join(domain[-2:])
    return domain

Make generator for read own base by batch

In [7]:
from __future__ import generators    # needs to be at the top of your module

def ResultIter(cursor, arraysize=100000):
    'An iterator that uses fetchmany to keep memory usage down'
    while True:
        results = cursor.fetchmany(arraysize)
        if not results:
            break
        for result in results:
            yield result

Make diff with own base

In [12]:
def make_diff(domains):
    start = dt.now()
    my_set = set(domains)
    cursor.execute("SELECT domain FROM result ORDER BY domain;")
    n = 0
    for result in ResultIter(cursor):
        my_set.difference_update(set(result))
    print(len(my_set))
    print(dt.now() - start)
    return my_set

## Download ru, rf, su domain from reg.ru

In [4]:
start = dt.now()
my_file = ''
for file in os.listdir(path + 'reg_ru/'):
    if '.pkl' in file:
        my_file = path + 'reg_ru/' + file
        break
reg_df = pd.read_pickle(my_file)
reg_df = reg_df.domain
print(dt.now() - start)
print(reg_df.shape)
reg_df.head()

0:00:01.626089
(5836461,)


2013481           jahve.ru
5730307         anycast.su
5731894          artium.su
5734627         azimuth.su
5735249    banana-grief.su
Name: domain, dtype: object

In [13]:
reg_df = make_diff(reg_df)

4078909
0:03:18.142565


## Download top 1M domains from Alexa

In [7]:
# df = pd.read_csv('~/top10milliondomains.csv')
# df.head()
first_start = dt.now()
df =  pd.read_csv('http://s3.amazonaws.com/alexa-static/top-1m.csv.zip', compression='zip', index_col=['index'], names=['index', 'domain'])
print(df.head(2))
print(df.shape)
myset = set(df.domain.values)
del df
print(len(myset))
print(dt.now() - first_start)

            domain
index             
1       google.com
2      youtube.com
(932711, 1)
932711
0:00:05.247584


## Download top 1M domains from AWS

In [8]:
start = dt.now()
df =  pd.read_csv('http://s3-us-west-1.amazonaws.com/umbrella-static/top-1m.csv.zip', compression='zip', index_col=['index'], names=['index', 'domain'])
print(df.head(2))
print(df.shape)
myset2 = set(df.domain.values)
del df
print(len(myset2))
print(dt.now() - start)

                       domain
index                        
1                 netflix.com
2      api-global.netflix.com
(1000000, 1)
1000000
0:00:09.287192


Make 1 dataset from Alexa and AWS

In [9]:
myset.update(myset2)
all_top = len(myset)
print(len(myset))

1828376


Search domains that not in our base

In [10]:
myset = make_diff(myset)`

0:02:44.236285


Make first levels domains 

In [11]:
start = dt.now()
myset = [get_top_level_domain(i) for i in myset]
myset = set(myset)
print(len(myset))
df = pd.DataFrame()
print(dt.now()-start)

489504
0:00:12.478858


Search first level domains that not in our base

In [10]:
myset = make_diff(myset)

Make DataFrame from all domains that not in our base

In [14]:
df = pd.DataFrame(reg_df)

In [15]:
df.shape

(4078909, 1)

Make small report

In [15]:
no_domain = df.shape[0]
exist_domain = all_top - no_domain
percent_no = no_domain/all_top*100
percent_exist = 100 - percent_no
print('В топе было {} доменов. Из них в базе есть {} ({}), нет {} ({})'\
      .format(all_top, exist_domain, percent_exist, no_domain, percent_no))

В топе было 1828376 доменов. Из них в базе есть 1440890 (78.8070943832122), нет 387486 (21.192905616787794)


In [16]:
print(dt.now() - first_start)

0:05:50.482709


Domains to format for crawler

In [16]:
df[0] = df[0].apply(lambda r: 'http://'+r)

In [17]:
df.head()

,0
0,http://rosservicelift.ru
1,http://mobilart.ru
2,http://udlinit-gazel.ru
3,http://olympic-unity.ru
4,http://moplay.ru


Save DataSet

In [18]:
file_name = 'reg_ru_domain_not_in_base.csv'
df.to_csv('~/{}'.format(file_name), sep='\t', index=False, header=False)

Put file on forest

In [19]:
import requests

In [20]:
put_path = "https://forest1.getaura.ru/feeds/experimental/{}".format(file_name)
resp = requests.put(put_path, data=open(path + '{}'.format(file_name), 'rb'))
if not resp.ok:
    print (resp.text)

In [21]:
put_path

'https://forest1.getaura.ru/feeds/experimental/reg_ru_domain_not_in_base.csv'